<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/3_5_pipeline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importamos las librerias

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


# Metricas
from sklearn.metrics import accuracy_score

# importing pipes for making the Pipe flow
from sklearn.pipeline import Pipeline



In [ ]:
# pasos 1. PC, 2 estandarización, 3. decision tree

pipe = Pipeline([('paso_pca', PCA(n_components = 2)), ('paso_estandarizacion', StandardScaler()),  ('paso_decision_tree', DecisionTreeClassifier())], verbose = True)

# COn  fit aplicamos todo

pipe.fit(X_train, y_train)


print(accuracy_score(y_test, pipe.predict(X_test)))

[Pipeline] .......... (step 1 of 3) Processing paso_pca, total=   0.0s
[Pipeline]  (step 2 of 3) Processing paso_estandarizacion, total=   0.0s
[Pipeline]  (step 3 of 3) Processing paso_decision_tree, total=   0.0s
0.8933333333333333


In [ ]:
pipe = Pipeline([('paso_decision_tree', DecisionTreeClassifier())], verbose = True)

# fitting the data in the pipeline
pipe.fit(X_train, y_train)

print(accuracy_score(y_test, pipe.predict(X_test)))

[Pipeline]  (step 1 of 1) Processing paso_decision_tree, total=   0.0s
0.9466666666666667


In [ ]:
# cargamos el dataset de prueba

iris = datasets.load_iris()
X = iris.data
y = iris.target

#Partimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50)

# paso 1 estandarización, paso 2 clasificadro

pipe = Pipeline([('paso_estandarizacion', StandardScaler()), ('paso_decision_tree', DecisionTreeClassifier())], verbose = True)

# Con fit aplica todos los pasos

pipe.fit(X_train, y_train)

# mostramos los resultados
print(accuracy_score(y_test, pipe.predict(X_test)))

[Pipeline]  (step 1 of 2) Processing paso_estandarizacion, total=   0.0s
[Pipeline]  (step 2 of 2) Processing paso_decision_tree, total=   0.0s
0.96
